# Evaluating pipline

In [41]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv

os.chdir('../app')

load_dotenv()

from ragas.evaluation import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, AnswerRelevancy, ContextPrecision
from ragas.run_config import RunConfig

from components.router import Router
from components.llm import LLM
from components.retriever import Retriever
from workflow.rag_workflow import RAGWorkflow

## Загрузка RAG

In [42]:
llm = LLM(
    model='llama-3-8b-instruct-8k',
    model_provider='openai',
    api_url='https://llama3gpu.neuraldeep.tech/v1',
    api_key='API_KEY',
)

retriever = Retriever(
    artifacts_path='./artifacts',
    dataframe_path='all_data.csv',
    embedding_model_name='all-mpnet-base-v2',
)

router = Router(
    artifacts_path='./artifacts',
    router_config_path='router_config.json',
    index_router_path='index_router.pickle'
)

workflow = RAGWorkflow(
   llm=llm,
   router=router,
   retriever=retriever
)

2025-04-06 15:58:58 - sentence_transformers.SentenceTransformer - INFO - SentenceTransformer.py:211 - __init__() - Use pytorch device_name: mps
2025-04-06 15:58:58 - sentence_transformers.SentenceTransformer - INFO - SentenceTransformer.py:219 - __init__() - Load pretrained SentenceTransformer: all-mpnet-base-v2
2025-04-06 15:59:03 - semantic_router.utils.logger - INFO - base.py:114 - from_file() - Loading route config from ./artifacts/router_config.json
2025-04-06 15:59:03 - semantic_router.utils.logger - WARNING - base.py:442 - _get_index() - No index provided. Using default LocalIndex.


## Подготовка датасета для проверки RAGAS

In [46]:
data = pd.read_csv('../tests/clear_data.csv')
validation_set = data[['question', 'answer']].rename(columns={'answer': 'ground_truth'}).sample(10)
validation_set.head()

,question,ground_truth
115,Как чистить уши французскому бульдогу?,Используйте специальный лосьон для чистки ушей...
21,Какие требования предъявляются к древесине?,"Древесина должна быть высокого качества, обраб..."
168,"Что делать, если кожа шелушится?",Увлажняйте ее и используйте мягкие эксфолианты.
196,Что такое постакне?,"Это следы после прыщей: рубцы, пятна, неровности."
327,Можно ли уволить работника без его согласия?,"Да, если есть законные основания, например, со..."


In [47]:
from tqdm import tqdm
from datasets import Dataset 

def create_data_for_ragas(workflow, eval_data):
    rag_dataset = []
    for _, row in tqdm(eval_data.iterrows()):
        result = workflow.graph.invoke(
            input={
            "question": row['question'],
            "messages": []
            },
            config={
                "configurable": {
                    "thread_id": 'qwdhou',
                }
            } 
        )

        rag_dataset.append(
            {
                "question": row['question'],
                "answer": result['messages'][-1].content,
                "retrieved_contexts": result['context'],
                "ground_truth": row['ground_truth']
            }
        )

    rag_df = pd.DataFrame(rag_dataset)
    rag_eval_dataset = Dataset.from_pandas(rag_df)

    return rag_eval_dataset

In [48]:
qa_ragas_baseline = create_data_for_ragas(workflow, validation_set)

Batches: 100%|██████████| 1/1 [00:00<00:00,  4.21it/s]
2025-04-06 16:00:45 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://llama3gpu.neuraldeep.tech/v1/chat/completions "HTTP/1.1 200 OK"
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.02it/s]
2025-04-06 16:00:47 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://llama3gpu.neuraldeep.tech/v1/chat/completions "HTTP/1.1 200 OK"
Batches: 100%|██████████| 1/1 [00:00<00:00, 13.92it/s]
2025-04-06 16:00:48 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://llama3gpu.neuraldeep.tech/v1/chat/completions "HTTP/1.1 200 OK"
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.87it/s]
2025-04-06 16:00:49 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://llama3gpu.neuraldeep.tech/v1/chat/completions "HTTP/1.1 200 OK"
Batches: 100%|██████████| 1/1 [00:00<00:00, 29.31it/s]
2025-04-06 16:00:50 - httpx - INFO - _cli

In [49]:
qa_ragas_baseline[0]

{'question': 'Как чистить уши французскому бульдогу?',
 'answer': 'Используйте специальный лосьон для чистки ушей и ватные диски.',
 'retrieved_contexts': ['Как чистить уши французскому бульдогу? Используйте специальный лосьон для чистки ушей и ватные диски.',
  'Почему у французского бульдога большие уши? Это результат селекции, такие уши стали визитной карточкой породы.',
  'Какой рост у французского бульдога? Рост в холке — около 25–35 см.',
  'Можно ли путешествовать с французским бульдогом? Да, но важно учитывать их чувствительность к перепадам температуры.',
  'Как выглядит французский бульдог? Это компактная собака с мускулистым телом, короткой шерстью, большими ушами и характерной плоской мордой.'],
 'ground_truth': 'Используйте специальный лосьон для чистки ушей и ватные диски.'}

## Оценка метрик
Метрики:
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/context_precision/?h=context)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/context_recall/?h=context)
- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/faithfulness/?h=faithfulness)

In [50]:
my_config = RunConfig(max_workers=2, timeout=180, max_wait=120)
evaluator = LangchainLLMWrapper(llm.model)
embedding = LangchainEmbeddingsWrapper(retriever) # такой вариант не работает в логах видно что нет атрибута 

Дальше в логах оценки метрик видна ошибка для модели векторизации. Так как LangchainEmbeddingsWrapper(класс RAGAS) не умет работать с Semantic Transformer

2025-04-06 15:26:44 - ragas.executor - ERROR - executor.py:104 - wrapped_callable_async() - Exception raised in Job[3]: AttributeError('Retriever' object has no attribute 'embed_query')


In [51]:
baseline_result = evaluate(  
  dataset=qa_ragas_baseline,  
  metrics=[  
      ContextPrecision(),  
      LLMContextRecall(),  
      Faithfulness(),   
    ],
  llm=evaluator,
  run_config=my_config,
  )

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]2025-04-06 16:01:13 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://llama3gpu.neuraldeep.tech/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-06 16:01:13 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://llama3gpu.neuraldeep.tech/v1/chat/completions "HTTP/1.1 200 OK"
Evaluating:   3%|▎         | 1/30 [00:01<00:53,  1.83s/it]2025-04-06 16:01:14 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://llama3gpu.neuraldeep.tech/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-06 16:01:15 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://llama3gpu.neuraldeep.tech/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-06 16:01:15 - httpx - INFO - _client.py:1740 - _send_single_request() - HTTP Request: POST https://llama3gpu.neuraldeep.tech/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-06 16:01:16 - httpx - INFO - 

In [52]:
# вывод оценок полученных на основе датасета
baseline_result

{'context_precision': 0.9500, 'context_recall': 1.0000, 'faithfulness': 0.9667}